--- Day 5: Print Queue ---
Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
53 is correctly fourth because it is before page number 29 (53|29).
29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

75,47,61,53,29
97,61,53,29,13
75,29,13
These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?

In [1]:
with open("i5.txt") as f:
    rules = {}
    updates = []
    is_rule = True
    for line in f.readlines():
        if line == "\n":
            is_rule = False
            continue
        if is_rule:
            parts = line.strip().split("|")
            prev_page = int(parts[0])
            later_page = int(parts[1])
            if prev_page not in rules:
                rules[prev_page] = []
            rules[prev_page].append(later_page)
        else:
            updates.append([int(x) for x in line.strip().split(",")])
    
rules, updates


({98: [43,
   99,
   91,
   24,
   97,
   39,
   62,
   35,
   85,
   34,
   66,
   89,
   11,
   38,
   37,
   57,
   36,
   75,
   47,
   86,
   14,
   42,
   73,
   68],
  91: [38,
   97,
   39,
   56,
   14,
   99,
   36,
   35,
   66,
   11,
   43,
   42,
   73,
   75,
   49,
   57,
   86,
   62,
   34,
   89,
   29,
   68,
   85,
   53],
  11: [56,
   72,
   55,
   77,
   34,
   31,
   41,
   94,
   67,
   99,
   93,
   38,
   49,
   82,
   33,
   43,
   85,
   59,
   66,
   22,
   86,
   53,
   29,
   68],
  36: [34,
   73,
   66,
   49,
   31,
   33,
   99,
   62,
   11,
   38,
   97,
   85,
   89,
   39,
   57,
   14,
   68,
   43,
   56,
   86,
   42,
   55,
   53,
   29],
  22: [69,
   37,
   98,
   36,
   45,
   35,
   96,
   91,
   42,
   94,
   19,
   72,
   75,
   63,
   97,
   39,
   76,
   89,
   87,
   24,
   73,
   47,
   57,
   23],
  69: [97,
   85,
   75,
   42,
   45,
   39,
   47,
   34,
   89,
   62,
   14,
   37,
   36,
   99,
   66,
   24,
   91,
   68,
   11

In [103]:
rules.get(49)

[82,
 33,
 19,
 22,
 67,
 87,
 45,
 96,
 47,
 72,
 53,
 63,
 76,
 98,
 41,
 55,
 23,
 93,
 31,
 77,
 69,
 94,
 59,
 56]

In [104]:
len(updates)

193

In [105]:
def is_correct(update):
    for i, page in enumerate(update):
        page_rules = rules.get(page)
        if not page_rules: continue # no rules for page
        # check if all previous pages are not in page rules to be behind
        for prev_page in update[:i]:
            if prev_page in page_rules:
                #print(f"{prev_page=} is before {page=}, but must be behind {page_rules=}")
                return False
    return True

middle_page_sum = 0
for update in updates:
    if is_correct(update):
        middle_page = update[len(update)//2]
        middle_page_sum += middle_page
        
middle_page_sum

5329

--- Part Two ---
While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

75,97,47,61,53 becomes 97,75,47,61,53.
61,13,29 becomes 61,29,13.
97,13,75,29,47 becomes 97,75,47,29,13.
After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

Answer: 
 

In [ ]:

def order(update):
    for i, page in enumerate(update):
        page_rules = rules.get(page)
        for j, prev in enumerate(update[:i]):
            if prev in page_rules:
                update[i], update[j] = update[j], update[i] # dont know exactly why thats enough
                return order(update) # try again - is it just luck that this works?
        
    return update

# for update in updates:
#     print(is_correct(order(update)), update)

In [107]:

unorderded_sum = 0
for i, update in enumerate(updates):
    if not is_correct(update):
        print(i, update)
        ordered = order(update)
        
        unorderded_sum += ordered[len(update)//2]
        
unorderded_sum


0 [98, 37, 47, 35, 22, 73, 76, 36, 67, 39, 69, 82, 45]
4 [37, 98, 86, 23, 42, 91, 96]
6 [34, 29, 77, 33, 89, 55, 97, 62, 11]
9 [93, 59, 98, 94, 55, 37, 82, 47, 19, 33, 22, 24, 96, 72, 67, 77, 23]
10 [23, 35, 63, 37, 69, 98, 47, 36, 96, 82, 19, 91, 57, 39, 89, 45, 24, 22, 94]
14 [72, 35, 93, 87, 47, 96, 82, 41, 37, 45, 23, 76, 98, 24, 67, 63, 59, 75, 19, 94, 69, 22, 77]
15 [87, 72, 91, 67, 24, 76, 47, 57, 75, 22, 73, 35, 19]
16 [36, 75, 98, 73, 24, 47, 87, 76, 89, 45, 96, 94, 35, 39, 63, 69, 57, 42, 22, 72, 23]
18 [49, 86, 75, 29, 68, 39, 35, 43, 91, 73, 38, 14, 57, 97, 34, 66, 11, 62, 42, 53, 85, 36, 89]
19 [76, 37, 47, 94, 93, 19, 45, 91, 22, 36, 77, 69, 98, 87, 82, 75, 41]
21 [37, 94, 98, 45, 19, 87, 35, 93, 72, 39, 77, 22, 63, 47, 23, 24, 91, 69, 75, 96, 36]
23 [41, 93, 82, 94, 31, 22, 96, 72, 59, 38, 76, 77, 67, 56, 33, 19, 63, 99, 29]
27 [69, 96, 91, 45, 87, 22, 72, 55, 67, 23, 94, 59, 76, 77, 24, 47, 75, 93, 82, 41, 98, 19, 37]
29 [85, 86, 38, 56, 29, 82, 76, 43, 31, 72, 93, 99, 

5833